In [0]:
import tensorflow as tf
# todo esto de bert usa librerias tensorflow.XXX , y es todo re sensible en compatibilidades

In [0]:
from google.colab import files


In [0]:
# ## run just once: ### 
# !unzip data.zip
# !unzip -o neural.zip
# # Tokenizador de Google para Bert
# !wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
# !pip install sentencepiece

In [0]:
# Load BERT from the Tensorflow Hub
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)


In [0]:
import pandas as pd
import numpy as np
import re

from neural.core.helpers_bert import bert_encode
from neural.core.helpers_bert import submission_bert

from neural.bert_architecture import build_model

In [0]:
RUN_SANITY = False
NEW_RANDOM_SPLIT = True
LOAD_PRETRAINED_VECS = False # False no genera el dict de embeddings (xq tarda unos min)

In [0]:
if NEW_RANDOM_SPLIT:
  semilla = np.random.randint(1,9999)
else:
  semilla = 800
rng = np.random.RandomState(semilla)
print(semilla)

In [0]:
import helpers_models as hm
datos_raw = hm.read_tagged_data()
datos = hm.clean_tagged_data(datos_raw)
X = datos['text']
y = datos['target']
X_train, X_val, y_train, y_val = hm.split_data(X, y, 0.2, rng)

In [0]:
# Thanks to https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub
# Load tokenizer from the bert layer
# We will use the official tokenization script created by the Google team
import tokenization

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
# Encode the text into tokens, masks, and segment flags

train_input = bert_encode(X_train, tokenizer, max_len=160)
train_labels = y_train

val_input = bert_encode(X_val, tokenizer, max_len=160)
val_labels = y_val

In [0]:
print(tf.__version__)

In [0]:
params = {
    'model_id': 'model_BERT.h5',
    'max_len': 160,
    'epochs': 5,
    'batch_size': 16
}

In [0]:
# Build BERT model with my tuning
model_BERT = build_model(bert_layer, params['max_len'])
model_BERT.summary()

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(params['model_id'] , monitor='val_accuracy', save_best_only=True)

train_history = model_BERT.fit(
    train_input, train_labels,
    validation_data=(val_input, val_labels),
    epochs = params['epochs'], # recomended 3-5 epochs
    callbacks=[checkpoint],
    batch_size = params['batch_size']
)

In [0]:
submission_bert(params['model_id'], tokenizer)


In [0]:
files.download(re.sub('.h5', '',params['model_id']) + '_submission.csv')